## Особенности данных:

1. сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
2. поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
3. продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
4. если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается

Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

Note: обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр format

Пример содержания итогового файла (колонка difference это разница между START_DTTM и END_DTTM):

ACT_DTTM | FILIAL_ID |difference
---------|-----------|-----------
20-03-2020 14:59 | 1 | 255 days 09:01:00

В папке subsid находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

In [1]:
import pandas as pd

tm_sales_1 = pd.read_csv('./subsid/tm_sales_1.csv', sep=';')
tm_sales_2 = pd.read_csv('./subsid/tm_sales_2.csv', sep=';')
tm_sales_3 = pd.read_csv('./subsid/tm_sales_3.csv', sep=';')

prod_activation_logs = pd.read_csv('./subsid/prod_activations_logs.csv', sep=';')
prod_activation_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43


In [2]:
tm_sales = pd.concat([tm_sales_1, tm_sales_2, tm_sales_3], ignore_index=True)
tm_sales.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,NaN,4,7421,12-03-2020 11:25


### Задание 1

Cотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить

*Если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается*

In [3]:
tm_sales.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,NaN,4,7421,12-03-2020 11:25


In [4]:
tm_sales = tm_sales.dropna(subset=['SUBS_ID'])

In [5]:
tm_sales.loc[~tm_sales['SUBS_ID'].str.startswith('id')]

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
1,7646509,5,6431,19-03-2020 13:00
5,8362218,7,9879,05-03-2020 11:42
6,2185490,2,3210,16-03-2020 16:28


In [6]:
tm_sales.loc[~tm_sales['SUBS_ID'].str.startswith('id'),
             'SUBS_ID'] = 'id' + tm_sales.loc[~tm_sales['SUBS_ID'].str.startswith('id'), 'SUBS_ID']
tm_sales.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,id8362218,7,9879,05-03-2020 11:42


In [7]:
sales_logs = prod_activation_logs.merge(tm_sales, on=['SUBS_ID', 'PROD_ID'])
sales_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00,1,20-03-2020 14:59
1,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03,5,19-03-2020 13:00
2,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00,3,17-03-2020 10:17
3,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00,2,16-03-2020 16:28
4,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42,6,15-03-2020 14:21


In [8]:
sales_logs.dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
FILIAL_ID      int64
ACT_DTTM      object
dtype: object

In [9]:
sales_logs['START_DTTM'] = pd.to_datetime(sales_logs['START_DTTM'])
sales_logs['END_DTTM'] = pd.to_datetime(sales_logs['END_DTTM'], format='%d-%m-%Y %H:%M')
sales_logs.dtypes


SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
FILIAL_ID              int64
ACT_DTTM              object
dtype: object

In [10]:
sales_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,19-03-2020 13:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21


In [11]:
sales_logs['difference'] = sales_logs['END_DTTM'] - sales_logs['START_DTTM']
sales_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,19-03-2020 13:00,0 days 00:03:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00


In [12]:
result = sales_logs.loc[sales_logs['difference'] > pd.Timedelta(5, 'm')]
result.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00


In [13]:
result['SUBS_ID'].sort_values().values

array(['id2185490', 'id4651830', 'id5416547', 'id7642700'], dtype=object)

In [14]:
result.to_csv('result.csv', index=False)

Ответ: id2185490, id4651830, id5416547, id7642700